In [3]:
import numpy as np
from PIL import Image
import time
from numba import cuda

# Загрузка изображения 
def load_image(filepath):
    image = Image.open(filepath).convert("L")  # Преобразование цветного изображение в серое
    img_array = np.array(image, dtype=np.uint8)
    return img_array

def save_image(img_array, filepath):
    img_array = np.clip(img_array, 0, 255)  # Обрезаем значения пикселей 
    image = Image.fromarray(img_array.astype(np.uint8))  # Преобразуем в uint8
    image.save(filepath)

# CUDA ядро для медианного фильтра
@cuda.jit
def median_filter(input_image, output_image, width, height):
    x, y = cuda.grid(2)
    if x >= 1 and x < width - 1 and y >= 1 and y < height - 1:
        window = np.zeros(9, dtype=np.uint8)
        idx = 0
        # Собираем окно 3x3
        for i in range(-1, 2):
            for j in range(-1, 2):
                window[idx] = input_image[y + i, x + j]
                idx += 1
        # Сортируем окно для нахождения медианы
        window.sort()
        # Устанавливаем медиану
        output_image[y, x] = window[4]

def apply_median_filter(img_array):
    height, width = img_array.shape
    # Создаем выходной массив
    output_image = np.zeros_like(img_array)
    # Переносим данные на GPU
    d_input = cuda.to_device(img_array)
    d_output = cuda.to_device(output_image)
    # Настройка сетки и блоков
    threads_per_block = (16, 16)
    blocks_per_grid_x = (width + (threads_per_block[0] - 1)) // threads_per_block[0]
    blocks_per_grid_y = (height + (threads_per_block[1] - 1)) // threads_per_block[1]
    # Запуск CUDA ядра
    median_filter[(blocks_per_grid_x, blocks_per_grid_y), threads_per_block, (d_input, d_output, width, height)]
    # Копируем результат обратно на CPU
    return d_output.copy_to_host()

if __name__ == "__main__":
    input_path = "input.bmp" # Исходное изображение - изображение с шумом
    output_path = "output.bmp" # Выходное изображение
    # Загрузка и обработка изображения
    img_array = load_image(input_path)
    # Измеряем время начала обработки
    start = time.time()
    # Применение медианного фильтра с использованием CUDA ядра
    output_img_array = apply_median_filter(img_array)
    # Сохранение результата
    save_image(output_img_array, output_path)
    print(f"Output image saved as {output_path}")
    print(f"Time taken: {time.time() - start:.3f} seconds")

Output image saved as output.bmp
Time taken: 0.002 seconds
